# 在函数中定义函数

In [1]:
def hi(name="yasoob"): 
    print("now you are inside the hi() function")
    
    def greet():
        return "now you are in the greet() function"
    
    def welcome():
        return "now you are in the welcomde() funcciton"
    
    print(greet())
    print(welcome())
    print("now you are back in the hi() function")

hi()
greet()

now you are inside the hi() function
now you are in the greet() function
now you are in the welcomde() funcciton
now you are back in the hi() function


NameError: name 'greet' is not defined

# 将函数作为参数传给另一个函数

In [3]:
def hi():
    return "hi yasoob!"

def doSomethingBeforeHi(func):
    print("I am doing some boring work before executing hi()")
    print(func())
    
doSomethingBeforeHi(hi)

I am doing some boring work before executing hi()
hi yasoob!


# 创建一个装饰器

In [4]:
# 装饰器其实就是将一个函数做一个包装，返回一个呗包装后的函数

def a_new_decorator(a_func):
    
    def wrapTheFunction():
        print("I am doing some boring work before executing a_func()")
        
        a_func()
        
        print("I am doing some boring work after excuting a_func()")
        
    return wrapTheFunction

def a_function_requiring_decoration():
    print("I am the function which needs some decoration to remove my foul smell")
    
a_function_requiring_decoration()

a_function_requiring_decoration = a_new_decorator(a_function_requiring_decoration)

a_function_requiring_decoration()

I am the function which needs some decoration to remove my foul smell
I am doing some boring work before executing a_func()
I am the function which needs some decoration to remove my foul smell
I am doing some boring work after excuting a_func()


In [5]:
@a_new_decorator
def a_function_requring_decoration():
    print("I am the function which needs some decoration to remove my foul smell")

a_function_requiring_decoration()

# 所以使用@a_new_decorator其实就是下述语句的缩写
# a_function_requiring_decoration = a_new_decorator(a_function_requiring_decoration)

I am doing some boring work before executing a_func()
I am the function which needs some decoration to remove my foul smell
I am doing some boring work after excuting a_func()


In [6]:
# 这里其实有个问题，函数被warpTheFunction替代了。它重写了原函数的名字和注释文档。我们可以用一个简单的函数来解决这个问题。
print(a_function_requiring_decoration.__name__)

wrapTheFunction


# functools.wraps

In [7]:
# 使用该函数来访问被包装的函数
from functools import wraps

def a_new_decorator(a_func):
    @wraps(a_func)
    def wrapTheFunction():
        print("I am doing some boring work before executing a_func()")
        a_func()
        print("I am doing some boring work after executing a_func()")
        
    return wrapTheFunction

@a_new_decorator
def a_function_requiring_decoration():
    """Hey yo! Decorate me!"""
    print("I am the function which needs some decoration to "
          "remove my foul smell")
print(a_function_requiring_decoration.__name__)

# 注意：@wraps接受一个函数来进行装饰，并加入了复制函数名称、注释文档、参数列表等等的功能。
# 这可以让我们在装饰器里面访问在装饰之前的函数的属性。

a_function_requiring_decoration


# 装饰器使用的蓝本规范

In [ ]:
from functools import wraps
def decorator_name(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        if not can_run:
            return "Function will not run"
        return f(*args, **kwargs)
    return decorated
 
@decorator_name
def func():
    return("Function is running")
 
can_run = True
print(func())
# Output: Function is running
 
can_run = False
print(func())
# Output: Function will not run

# 装饰器的使用场景
## 授权(Authorization)

In [ ]:
# 装饰器能有助于检查某个人是否被授权去使用一个web应用的端点(endpoint)。它们被大量使用Flask和Django web框架中。
# 这里是一个例子来使用基于装饰器的授权：
from functools import wraps
 
def requires_auth(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        auth = request.authorization
        if not auth or not check_auth(auth.username, auth.password):
            authenticate()
        return f(*args, **kwargs)
    return decorated

# 日志（Logging）

In [8]:
#日志是装饰器运用的另一个亮点。这是个例子：
from functools import wraps
 
def logit(func):
    @wraps(func)
    def with_logging(*args, **kwargs):
        print(func.__name__ + " was called")
        return func(*args, **kwargs)
    return with_logging
 
@logit
def addition_func(x):
   """Do some math."""
   return x + x
 
 
result = addition_func(4)
# Output: addition_func was called

addition_func was called


# 带参数的装饰器

In [ ]:
from functools import wraps
 
def logit(logfile='out.log'):
    def logging_decorator(func):
        @wraps(func)
        def wrapped_function(*args, **kwargs):
            log_string = func.__name__ + " was called"
            print(log_string)
            # 打开logfile，并写入内容
            with open(logfile, 'a') as opened_file:
                # 现在将日志打到指定的logfile
                opened_file.write(log_string + '\n')
            return func(*args, **kwargs)
        return wrapped_function
    return logging_decorator
 
@logit()
def myfunc1():
    pass
 
myfunc1()
# Output: myfunc1 was called
# 现在一个叫做 out.log 的文件出现了，里面的内容就是上面的字符串
 
@logit(logfile='func2.log')
def myfunc2():
    pass
 
myfunc2()
# Output: myfunc2 was called
# 现在一个叫做 func2.log 的文件出现了，里面的内容就是上面的字符串

# 装饰器类

In [ ]:
from functools import wraps
 
class logit(object):
    def __init__(self, logfile='out.log'):
        self.logfile = logfile
 
    def __call__(self, func):
        @wraps(func)
        def wrapped_function(*args, **kwargs):
            log_string = func.__name__ + " was called"
            print(log_string)
            # 打开logfile并写入
            with open(self.logfile, 'a') as opened_file:
                # 现在将日志打到指定的文件
                opened_file.write(log_string + '\n')
            # 现在，发送一个通知
            self.notify()
            return func(*args, **kwargs)
        return wrapped_function
 
    def notify(self):
        # logit只打日志，不做别的
        pass

In [ ]:
@logit()
def myfunc1():
    pass

In [ ]:
class email_logit(logit):
    '''
    一个logit的实现版本，可以在函数调用时发送email给管理员
    '''
    def __init__(self, email='admin@myproject.com', *args, **kwargs):
        self.email = email
        super(email_logit, self).__init__(*args, **kwargs)
 
    def notify(self):
        # 发送一封email到self.email
        # 这里就不做实现了
        pass